In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
# import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
#plt.style.use('fivethirtyeight')
import xgboost as xgb
import sklearn
import random
from NecessaryModules.getData import getData

from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
# from NecessaryModules.splitData import split_data

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_absolute_error

In [3]:
filename = '../shot_listed_dataset/CoverTypeNewNormalized.csv'
dataset=getData(filename).values.tolist()
random.shuffle(dataset)
dataset=pd.DataFrame(dataset) #from dataset_list to dataset_dataframe after suffling
# print(dataset)

   Elevation    Aspect     Slope  Horizontal_Distance_To_Hydrology  \
0   0.850622  0.105556  0.150943                          0.045351   
1   0.879149  0.163889  0.188679                          0.426304   
2   0.767116  0.327778  0.320755                          0.068027   
3   0.791494  0.338889  0.226415                          0.294785   
4   0.867220  0.575000  0.584906                          0.385488   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                       -0.003509                         0.631060   
1                        0.189474                         0.177358   
2                        0.038596                         0.259652   
3                       -0.077193                         0.085486   
4                        0.314035                         0.074020   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
0       0.866142        0.874016          0.548   
1       0.897638        0.866142          0.500   
2    

In [4]:
def split_data(data):
    # splitting the data
    
    msk = np.random.rand(len(data)) < 0.7
    yy = [54] #test title is 54 after suffleing the dataframe 
    train = data[msk]
    test = data[~msk]
    x_train = train[train.columns[:train.shape[1] - 1]]  # all columns except the last are x variables
    y_train = train[yy[0]].tolist()  # the last column as y variable
    x_test = test[test.columns[:test.shape[1] - 1]]
    y_test = test[yy[0]].tolist()  # the last column as y variable
    y_train = [0 if i == 7 else i for i in y_train]  # replace label 7 with 0
    y_test = [0 if i == 7 else i for i in y_test]  # replace label 7 with 0
    return x_train, y_train, x_test, y_test

# RFE implement

In [5]:
X=dataset.iloc[:,0:-1]
y=dataset.iloc[:,-1]

In [6]:
estimator = SVR(kernel="linear")
model = RFE(estimator, n_features_to_select=40, step=1)
rfeTrain = model.fit(X,y)
print(rfeTrain)

compressed_dataset_X= rfeTrain.transform(X)
print(type(compressed_dataset_X))
print(len(compressed_dataset_X))
print(compressed_dataset_X)


RFE(estimator=SVR(kernel='linear'), n_features_to_select=40)
<class 'numpy.ndarray'>
19229
[[0.54797718 1.         1.         ... 0.         0.         0.        ]
 [0.85892116 0.         0.         ... 1.         0.         0.        ]
 [0.50985477 1.         0.         ... 0.         0.         0.        ]
 ...
 [0.73236515 0.         0.         ... 0.         0.         0.        ]
 [0.64211618 0.         0.         ... 0.         0.         0.        ]
 [0.77567427 0.         0.         ... 0.         0.         0.        ]]


In [7]:
# from NecessaryModules.splitData import split_data
X_train, y_train, X_test, y_test = split_data(dataset)
print(len(X_train))

13462


In [8]:
compressed_dataset_X_train= rfeTrain.transform(X_train) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_train))
print(len(compressed_dataset_X_train))
print(compressed_dataset_X_train)

compressed_dataset_X_test= rfeTrain.transform(X_test) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_test))
print(len(compressed_dataset_X_test))
print(compressed_dataset_X_test)

<class 'numpy.ndarray'>
13462
[[0.54797718 1.         1.         ... 0.         0.         0.        ]
 [0.85892116 0.         0.         ... 1.         0.         0.        ]
 [0.50985477 1.         0.         ... 0.         0.         0.        ]
 ...
 [0.76374481 0.         0.         ... 0.         0.         0.        ]
 [0.73236515 0.         0.         ... 0.         0.         0.        ]
 [0.64211618 0.         0.         ... 0.         0.         0.        ]]
<class 'numpy.ndarray'>
5767
[[0.61877593 1.         0.         ... 0.         0.         0.        ]
 [0.49507261 1.         1.         ... 0.         0.         0.        ]
 [0.89859959 0.         0.         ... 0.         1.         0.        ]
 ...
 [0.54512448 1.         0.         ... 0.         0.         0.        ]
 [0.56405602 1.         0.         ... 0.         0.         0.        ]
 [0.77567427 0.         0.         ... 0.         0.         0.        ]]


# tune with SVC

In [9]:
# train the model with random parameter
from sklearn.svm import SVC

model_SVC = SVC(
        C=8.75,#Regularization parameter
        kernel="rbf", # maximum number of estimators at which boosting is terminated
        gamma=0.4,
        tol=0.483,
        decision_function_shape='ovo',
        
     )

model_SVC.fit(compressed_dataset_X_train, y_train)

SVC(C=8.75, decision_function_shape='ovo', gamma=0.4, tol=0.483)

In [10]:
# Finding mean absolute error
preds = model_SVC.predict(compressed_dataset_X_test)
preds = preds.astype(int)
mae = np.abs(y_test-preds).mean()
mae

0.7062597537714583

In [11]:
# Defining the best parameter with optuna
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold
K = 5 # stratified 5 fold cross validation, The folds are made by preserving the percentage of samples for each class.
skf = StratifiedKFold(n_splits = K, random_state = 42, shuffle = True)

train_tunned= np.array(compressed_dataset_X)
target_train_tunned= np.array(y)

oof_preds= np.zeros((len(train_tunned)))
abs_error_arr=[]
def objective_SVC(trial):

 for train_index, val_index in (skf.split(compressed_dataset_X,y)):
    train_X, valid_X = train_tunned[train_index], train_tunned[val_index]
    train_Y, valid_Y = target_train_tunned[train_index], target_train_tunned[val_index]
    
        
        
    param = {
        'C':trial.suggest_float("C",0,10), #For regularization    
        'kernel':trial.suggest_categorical('kernel',["linear", "poly", "rbf", "sigmoid"]),
        'degree':trial.suggest_int('degree',1,10),
        'gamma':trial.suggest_float('gamma',0,1),
        # 'tol':trial.suggest_float('tol',0,5),
        # 'coef0':trial.suggest_float('coed0',0,3),
        'decision_function_shape':trial.suggest_categorical('decision_function_shape',['ovo','ovr']),
        'random_state':trial.suggest_int('random_state',0,5),
        }
    model= SVC(**param)  
        
    model.fit(train_X,train_Y)
        
    preds = model.predict(valid_X)
    oof_preds[val_index] = preds # preds gives total 3846 predicted value for valid_X

      
 mae = mean_absolute_error(target_train_tunned, oof_preds)
        
 return mae

In [12]:
# this is the main optuna for optimizing a object-->"objective"
study_SVC = optuna.create_study(direction='minimize',study_name="SVC_tunning")

[I 2024-02-19 19:59:23,792] A new study created in memory with name: SVC_tunning


In [13]:

study_SVC.optimize(objective_SVC, n_trials=15) 
print('Number of finished trials:', len(study_SVC.trials))
print('Best trial:', study_SVC.best_trial.params)

[I 2024-02-19 20:00:08,560] Trial 0 finished with value: 1.032711009412866 and parameters: {'C': 9.080948513808536, 'kernel': 'sigmoid', 'degree': 10, 'gamma': 0.8874071226485558, 'decision_function_shape': 'ovo', 'random_state': 2}. Best is trial 0 with value: 1.032711009412866.
[I 2024-02-19 20:00:37,607] Trial 1 finished with value: 0.9990119090956368 and parameters: {'C': 0.5201437014827814, 'kernel': 'linear', 'degree': 9, 'gamma': 0.5152638911615832, 'decision_function_shape': 'ovr', 'random_state': 0}. Best is trial 1 with value: 0.9990119090956368.
[I 2024-02-19 20:01:18,802] Trial 2 finished with value: 0.982422382859223 and parameters: {'C': 9.577274041492931, 'kernel': 'sigmoid', 'degree': 6, 'gamma': 0.08764880879176506, 'decision_function_shape': 'ovo', 'random_state': 1}. Best is trial 2 with value: 0.982422382859223.
[I 2024-02-19 20:01:58,817] Trial 3 finished with value: 0.9699932393780227 and parameters: {'C': 8.219865046226168, 'kernel': 'sigmoid', 'degree': 1, 'gamm

Number of finished trials: 15
Best trial: {'C': 7.520533788272246, 'kernel': 'linear', 'degree': 6, 'gamma': 0.8219005763032301, 'decision_function_shape': 'ovo', 'random_state': 1}


In [14]:
#evaluate the trial
print("Number of finished trials: ", len(study_SVC.trials))
print("Best trial:")
trial = study_SVC.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials:  15
Best trial:
  Value:  0.8967184981018254
  Params: 
    C: 7.520533788272246
    kernel: linear
    degree: 6
    gamma: 0.8219005763032301
    decision_function_shape: ovo
    random_state: 1


# process the whole data

In [15]:
# filename_2 = '../shot_listed_dataset/Cover_type_whole_genuine_dataset_normalization.csv'
# dataset_2=getData(filename_2).values.tolist()
# random.shuffle(dataset_2)



In [16]:
# dataset_2=pd.DataFrame(dataset_2) #from dataset_list to dataset_dataframe after suffling
# print(dataset_2)

In [17]:
# X_2=dataset_2.iloc[:,0:-1]
# y_2=dataset_2.iloc[:,-1]
# X_train_2, y_train_2, X_test_2, y_test_2 = split_data(dataset_2)

In [18]:

# estimator = SVR(kernel="linear")
# model = RFE(estimator, n_features_to_select=30, step=1)
# rfeTrain_2 = model.fit(X_2,y_2)# --train-- pca with the "dataset" Dataframe
# print(rfeTrain_2)
# compressed_dataset_X_2 = rfeTrain_2.transform(X_2) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
# print(type(compressed_dataset_X_2))

In [19]:
# compressed_dataset_X_train_2= rfeTrain_2.transform(X_train_2) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
# print(type(compressed_dataset_X_train_2))
# print(len(compressed_dataset_X_train_2))
# print(compressed_dataset_X_train_2)

# compressed_dataset_X_test_2= rfeTrain_2.transform(X_test_2) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
# print(type(compressed_dataset_X_test_2))
# print(len(compressed_dataset_X_test_2))
# print(compressed_dataset_X_test_2)

# fit the SVM model

In [20]:
# Let us make predictions for each of the 5 models and find mean 
# of those predictions

from tqdm import tqdm
train_SVC = np.array(compressed_dataset_X_train)
target_train_SVC = np.array(y_train)

test_SVC = np.array(compressed_dataset_X_test) # or test_Ada = np.array(X)


test_preds_SVC = np.zeros((len(test_SVC)))
oof_preds_SVC = np.zeros((len(train_SVC)))

for train_index, val_index in tqdm(skf.split(compressed_dataset_X_train,y_train), total=5):
    train_X, valid_X = train_SVC[train_index], train_SVC[val_index]
    train_y, valid_y = target_train_SVC[train_index], target_train_SVC[val_index]
    svc_params = study_SVC.best_trial.params
    model_SVC_2 = SVC(**svc_params)
    model_SVC_2.fit(train_X, train_y)
    # for train accuracy
    preds = model_SVC_2.predict(valid_X)
    oof_preds_SVC[val_index] = preds
    # for test accuracy
    preds = model_SVC_2.predict(test_SVC)
    test_preds_SVC += (preds)/5
print(np.abs(oof_preds_SVC-target_train_SVC).mean())

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:22<00:00,  4.47s/it]

0.6924676868221661


In [21]:
from sklearn.metrics import accuracy_score
# valid_Acc_SVC=accuracy_score(target_train,oof_preds)
# print(valid_Acc_SVC)
target_test=np.array(y_test)
# another technique for calculate test accuracy
test_Acc_SVC=accuracy_score(target_test,test_preds_SVC.round())
print(test_Acc_SVC)

0.648344026356858


In [22]:
#validation accuracy
valid_Acc=accuracy_score(target_train_SVC,oof_preds_SVC)
print(valid_Acc)

0.6533204575843113


In [23]:
# train accuracy
train_Acc=model_SVC_2.score(compressed_dataset_X_train,y_train)
print(train_Acc)

0.6525033427425345


In [24]:
# test_accuracy
test_Acc=model_SVC_2.score(compressed_dataset_X_test,y_test)
print(test_Acc)

0.648344026356858
